In [1]:
import math
import pandas as pd
import numpy as np
from enum import Enum

class MessageKind(Enum):
    MEASURE = "MEASURE"
    GENREQ = "GENREQ"
    GENRESP = "GENRESP"
    
class ReqResKind(Enum):
    READ = "READ"
    COMMAND = "COMMAND"
    UNSET = "UNSET"
    
class AtkStatus(Enum):
    UNMOD = "UNMOD"
    COMPR = "COMPR"
    BLOCK = "BLOCK"
    
class EvilState(Enum):
    INACTIVE = "INACTIVE"
    MIDDLE = "MIDDLE"
    FULL = "FULL"
    
class Keys(Enum):
    ID = "id"
    MESSAGEKIND = "messageKind"
    REQRESKIND = "reqResKind"
    ATKSTATUS = "atkStatus"
    DATA = "data"
    EVILSTATE = "evilState"
    CREATIONTIME = "creationTime"
    TIMESTAMP = "timestamp"

class OutKeys(Enum):
    MITM = "MITM"
    SRM = "SRM"
    UC = "UC"
    UPS = "UPS"
    IMD = "IMD"
    MC = "MC"
    CC = "CC"
    
    
evilStateToVar = {
    EvilState.INACTIVE.value: 1,
    EvilState.MIDDLE.value: 2,
    EvilState.FULL.value: 3
}
    
# Simulation time limit in seconds
simTimeLimit = 120
# Number of runs
numRuns = 50
# Connection index (0 for server[0] etc...)
connIndex = 0
# Discretization time in seconds
deltaT = 5
# Number of levels for SRM and UC
numLevels = 4
# Time between consecutive measures (in seconds)
measureInterval = 4


relPath = "../logs/"
evilFileName = "evilClient.app[" + connIndex.__str__() + "].csv"
cliFileName = "client[0].app[" + connIndex.__str__() + "].csv"
serFileName = "server[" + connIndex.__str__() + "].app[0].csv"


In [2]:
for i in range(numRuns):
    complRelPath = relPath + "run" + i.__str__() + "/"
    evilDf = pd.read_csv(complRelPath + evilFileName)
    cliDf = pd.read_csv(complRelPath + cliFileName)
    evilDf = evilDf.drop(Keys.CREATIONTIME.value, axis=1)
    
    currT = 0
    MITMv = 0
    SRMv = 0
    UCv = 0
    MCv = 0
    CCv = 0
    outDf = pd.DataFrame(columns=[OutKeys.MITM.value, OutKeys.SRM.value, OutKeys.UC.value, OutKeys.MC.value, OutKeys.CC.value])
    while currT < simTimeLimit:
        evilWindowDf = evilDf[evilDf[Keys.TIMESTAMP.value].between(currT, currT + deltaT, inclusive="left")]
        cliWindowDf = cliDf[cliDf[Keys.TIMESTAMP.value].between(currT, currT + deltaT, inclusive="left")]
        evilTotNumWindow = evilWindowDf.shape[0]
        cliTotNumWindow = cliWindowDf.shape[0]
        if not evilWindowDf.empty:
            MITMv = evilStateToVar[evilWindowDf.iloc[-1][Keys.EVILSTATE.value]]
            numComprMeas = evilWindowDf[((evilWindowDf[Keys.MESSAGEKIND.value] == MessageKind.MEASURE.value) | 
                                            (evilWindowDf[Keys.REQRESKIND.value] == ReqResKind.READ.value)) & 
                                            (evilWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            numComprCmds = evilWindowDf[(evilWindowDf[Keys.REQRESKIND.value] == ReqResKind.COMMAND.value) & 
                                            (evilWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            SRMv = max(math.trunc((numComprMeas/evilTotNumWindow) * 4), SRMv)
            UCv = max(math.trunc((numComprCmds/evilTotNumWindow) * 4), UCv)
        if not cliWindowDf.empty:
            numComprMeas = cliWindowDf[((cliWindowDf[Keys.MESSAGEKIND.value] == MessageKind.MEASURE.value) | 
                                            (cliWindowDf[Keys.REQRESKIND.value] == ReqResKind.READ.value)) & 
                                            (cliWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            numComprCmds = cliWindowDf[(cliWindowDf[Keys.REQRESKIND.value] == ReqResKind.COMMAND.value) & 
                                            (cliWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            MCv = max(math.trunc((numComprMeas/cliTotNumWindow) * 4), MCv)
            CCv = max(math.trunc((numComprCmds/cliTotNumWindow) * 4), CCv)
        outDf.loc[len(outDf.index)] = [MITMv, SRMv, UCv, MCv, CCv]

        currT += deltaT
    print(outDf)
    outDf.to_csv("outTrain.csv", index=False)


    MITM  SRM  UC  MC  CC
0      0    0   0   0   0
1      1    0   0   0   0
2      1    0   0   0   0
3      1    0   0   0   0
4      2    0   0   0   0
5      2    2   0   2   0
6      2    2   0   2   0
7      2    2   0   2   0
8      2    2   0   2   0
9      3    2   1   2   0
10     3    2   1   2   0
11     3    2   1   2   0
12     3    2   1   2   0
13     3    2   1   2   0
14     3    2   1   2   0
15     3    2   1   2   0
16     3    2   1   2   0
17     3    2   1   2   0
18     3    2   1   2   0
19     3    2   1   2   0
20     3    2   1   2   0
21     3    2   1   2   0
22     3    2   1   2   0
23     3    2   1   2   0
    MITM  SRM  UC  MC  CC
0      0    0   0   0   0
1      1    0   0   0   0
2      1    0   0   0   0
3      2    0   0   0   0
4      2    3   0   2   0
5      2    3   1   2   0
6      3    3   1   2   0
7      3    3   1   2   0
8      3    3   1   2   0
9      3    3   1   2   0
10     3    3   1   2   0
11     3    3   1   2   0
12     3    

In [3]:
def getUnstProb(numMsg):
    q = 6
    k = 4
    return 1 / (1+math.exp((-(1/q)*numMsg)+k))

In [4]:
# Initialize columns labels
columns = []
for key in OutKeys:
    columns.append(key.value)
    for j in range(1, (simTimeLimit // deltaT)+1):
        columns.append(key.value + "_" + j.__str__())
outDf = pd.DataFrame(columns=columns)  

for i in range(numRuns):
    complRelPath = relPath + "run" + i.__str__() + "/"
    evilDf = pd.read_csv(complRelPath + evilFileName)
    cliDf = pd.read_csv(complRelPath + cliFileName)
    serDf = pd.read_csv(complRelPath + serFileName)
    evilDf = evilDf.drop(Keys.CREATIONTIME.value, axis=1)
    cliDf = cliDf.drop(Keys.CREATIONTIME.value, axis=1)
    serDf = serDf.drop(Keys.CREATIONTIME.value, axis=1)
    
    # Finds all the GENREQ without a GENRESP associated
    cliReqWoRes = cliDf.drop_duplicates(Keys.ID.value, keep=False)
    cliReqWoRes = cliReqWoRes[cliReqWoRes[Keys.MESSAGEKIND.value] == MessageKind.GENREQ.value]
    
    
    currT = 0
    
    MITMv = 0
    SRMv = 0
    UCv = 0
    MCv = 0
    CCv = 0
    UPSv = 0
    IMDv = 0
    
    totNumComprCmds = 0
    totNumComprMeas = 0
    
    lastMeasRecTime = -1
    
    while currT < simTimeLimit:
        evilWindowDf = evilDf[evilDf[Keys.TIMESTAMP.value].between(currT, currT + deltaT, inclusive="left")]
        cliWindowDf = cliDf[cliDf[Keys.TIMESTAMP.value].between(currT, currT + deltaT, inclusive="left")]
        serWindowDf = serDf[serDf[Keys.TIMESTAMP.value].between(currT, currT + deltaT, inclusive="left")]
        evilTotNumWindow = evilWindowDf.shape[0]
        cliTotNumWindow = cliWindowDf.shape[0]
        serTotNumWindow = serWindowDf.shape[0]
        # Analize attacker's log
        if not evilWindowDf.empty:
            MITMv = evilStateToVar[evilWindowDf.iloc[-1][Keys.EVILSTATE.value]]
            numComprMeas = evilWindowDf[((evilWindowDf[Keys.MESSAGEKIND.value] == MessageKind.MEASURE.value) | 
                                            (evilWindowDf[Keys.REQRESKIND.value] == ReqResKind.READ.value)) & 
                                            (evilWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            numComprCmds = evilWindowDf[(evilWindowDf[Keys.REQRESKIND.value] == ReqResKind.COMMAND.value) & 
                                            (evilWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            SRMv = max(math.trunc((numComprMeas/evilTotNumWindow) * numLevels), SRMv)
            if SRMv == numLevels: SRMv = numLevels - 1
            UCv = max(math.trunc((numComprCmds/evilTotNumWindow) * numLevels), UCv)
            if UCv == numLevels: UCv = numLevels - 1
        # Analize client's log
        if not cliWindowDf.empty:
            numComprMeas = cliWindowDf[((cliWindowDf[Keys.MESSAGEKIND.value] == MessageKind.MEASURE.value) | 
                                            (cliWindowDf[Keys.REQRESKIND.value] == ReqResKind.READ.value)) & 
                                            (cliWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            MCv = max(math.trunc((numComprMeas/cliTotNumWindow) * numLevels), MCv)
            if MCv == numLevels: MCv = numLevels - 1
            
            # Unstable message delivery if at least a MEASURE or a GENRESP has not been received
            measInWindow = cliWindowDf[((cliWindowDf[Keys.MESSAGEKIND.value] == MessageKind.MEASURE.value))]
            if not measInWindow.empty:
                lastMeasRecTime = measInWindow.iloc[-1][Keys.TIMESTAMP.value]
            if (lastMeasRecTime > 0 and (currT - lastMeasRecTime > measureInterval + 1)) or (not cliReqWoRes[cliReqWoRes[Keys.TIMESTAMP.value].between(currT, currT + deltaT, inclusive="left")].empty):
                IMDv = 1   
            
            totNumComprMeas += numComprMeas
        # Analize server's log
        if not serWindowDf.empty:
            numComprCmds = serWindowDf[(serWindowDf[Keys.REQRESKIND.value] == ReqResKind.COMMAND.value) & 
                                            (serWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            CCv = max(math.trunc((numComprCmds/serTotNumWindow) * numLevels), CCv)
            if CCv == numLevels: CCv = numLevels - 1
            
            totNumComprCmds += numComprCmds

        # Assign values
        currT += deltaT
        numSlice = currT // deltaT   
        # Calculates Unstable System probability
        if(UPSv == 0 and numSlice % 3 == 0 and np.random.uniform(0, 1) < getUnstProb(totNumComprMeas + totNumComprCmds)):
            UPSv = 1 
        outDf.at[i, OutKeys.MITM.value] = OutKeys.MITM.value + "0"
        outDf.at[i, OutKeys.MITM.value + "_" + numSlice.__str__()] = OutKeys.MITM.value + MITMv.__str__()
        outDf.at[i, OutKeys.SRM.value] = OutKeys.SRM.value + "0"
        outDf.at[i, OutKeys.SRM.value + "_" + numSlice.__str__()] = OutKeys.SRM.value + SRMv.__str__()
        outDf.at[i, OutKeys.UC.value] = OutKeys.UC.value + "0"
        outDf.at[i, OutKeys.UC.value + "_" + numSlice.__str__()] = OutKeys.UC.value + UCv.__str__()
        outDf.at[i, OutKeys.MC.value] = OutKeys.MC.value + "0"
        outDf.at[i, OutKeys.MC.value + "_" + numSlice.__str__()] = OutKeys.MC.value + MCv.__str__()
        outDf.at[i, OutKeys.CC.value] = OutKeys.CC.value + "0"
        outDf.at[i, OutKeys.CC.value + "_" + numSlice.__str__()] = OutKeys.CC.value + CCv.__str__()
        outDf.at[i, OutKeys.UPS.value] = OutKeys.UPS.value + "0"
        outDf.at[i, OutKeys.UPS.value + "_" + numSlice.__str__()] = OutKeys.UPS.value + UPSv.__str__()
        outDf.at[i, OutKeys.IMD.value] = OutKeys.IMD.value + "0"
        outDf.at[i, OutKeys.IMD.value + "_" + numSlice.__str__()] = OutKeys.IMD.value + IMDv.__str__()
    outDf.to_csv("outTrain.csv", index=False)
